In [15]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade import plotter
import numpy as np
from pyalgotrade.barfeed import quandlfeed
import datetime
from pyalgotrade.technical import ma
filename = "bank_nifty1.csv"
import pytz
firstDate = datetime.datetime(2012,5,30,9,15,0,0,pytz.utc)
endDate = datetime.datetime(2016,12,9,15,29,0,0,pytz.utc)

In [16]:
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [17]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade.technical import cross

In [18]:
class SMACrossOver(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed, 100000)
        self.__instrument = instrument
        self.__position = None
        # We'll use adjusted close values instead of regular close values.
        #self.setUseAdjustedValues(False)
        #self.__prices = feed[instrument].getPriceDataSeries()
        #self.__sma = ma.SMA(self.__prices, smaPeriod)
        self.__sma10 = ma.SMA(feed[instrument].getCloseDataSeries(), 10)
        self.__sma20 = ma.SMA(feed[instrument].getCloseDataSeries(), 20)

    def getSMA(self):
        return self.__sma10
    
    def getSMA(self):
        return self.__sma20
    
    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at %.2f" % (execInfo.getPrice()))
    

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at %.2f" % (execInfo.getPrice()))
        self.__position = None
        

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # If a position was not opened, check if we should enter a long position.
        bar = bars[self.__instrument]
        if self.__position is None :
            if cross.cross_above(self.__sma10,self.__sma20) > 0 and (bar.getDateTime().hour>10 and bar.getDateTime().hour<14):
                shares = int(self.getBroker().getCash() * 0.9 / bars[self.__instrument].getPrice())
                # Enter a buy market order. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, shares, True)
        # Check if we have to exit the position.
        elif not self.__position.exitActive() and cross.cross_below(self.__sma10,self.__sma20) > 0:
            self.__position.exitMarket()

In [19]:
feed = csvfeed.GenericBarFeed(1*Frequency.MINUTE,pytz.utc)
#feed = quandlfeed.Feed()
feed.setBarFilter(csvfeed.DateRangeFilter(firstDate,endDate))
feed.addBarsFromCSV('bank_nifty',filename)

crossover = SMACrossOver(feed,"bank_nifty")
crossover.run()

    

2012-05-30 11:02:00+00:00 strategy [INFO] BUY at 9530.00
2012-05-30 11:18:00+00:00 strategy [INFO] SELL at 9534.00
2012-05-30 11:22:00+00:00 strategy [INFO] BUY at 9537.50
2012-05-30 11:37:00+00:00 strategy [INFO] SELL at 9523.95
2012-05-30 11:51:00+00:00 strategy [INFO] BUY at 9529.75
2012-05-30 11:55:00+00:00 strategy [INFO] SELL at 9532.15
2012-05-30 11:59:00+00:00 strategy [INFO] BUY at 9536.10
2012-05-30 12:08:00+00:00 strategy [INFO] SELL at 9532.70
2012-05-30 12:13:00+00:00 strategy [INFO] BUY at 9529.35
2012-05-30 12:20:00+00:00 strategy [INFO] SELL at 9511.00
2012-05-30 12:56:00+00:00 strategy [INFO] BUY at 9518.00
2012-05-30 13:08:00+00:00 strategy [INFO] SELL at 9498.00
2012-05-30 13:26:00+00:00 strategy [INFO] BUY at 9492.75
2012-05-30 14:01:00+00:00 strategy [INFO] SELL at 9512.00
2012-05-31 11:16:00+00:00 strategy [INFO] BUY at 9395.10
2012-05-31 11:34:00+00:00 strategy [INFO] SELL at 9393.20
2012-05-31 11:43:00+00:00 strategy [INFO] BUY at 9398.00
2012-05-31 11:49:00+00:

2012-06-18 13:27:00+00:00 strategy [INFO] BUY at 9760.00
2012-06-18 13:39:00+00:00 strategy [INFO] SELL at 9743.45
2012-06-19 11:06:00+00:00 strategy [INFO] BUY at 9621.00
2012-06-19 11:47:00+00:00 strategy [INFO] SELL at 9688.00
2012-06-19 12:14:00+00:00 strategy [INFO] BUY at 9686.00
2012-06-19 12:27:00+00:00 strategy [INFO] SELL at 9670.20
2012-06-19 12:37:00+00:00 strategy [INFO] BUY at 9710.00
2012-06-19 13:23:00+00:00 strategy [INFO] SELL at 9766.50
2012-06-19 13:58:00+00:00 strategy [INFO] BUY at 9675.00
2012-06-19 14:36:00+00:00 strategy [INFO] SELL at 9760.00
2012-06-20 11:12:00+00:00 strategy [INFO] BUY at 9847.80
2012-06-20 11:19:00+00:00 strategy [INFO] SELL at 9827.00
2012-06-20 11:58:00+00:00 strategy [INFO] BUY at 9799.85
2012-06-20 12:14:00+00:00 strategy [INFO] SELL at 9799.10
2012-06-20 12:26:00+00:00 strategy [INFO] BUY at 9814.90
2012-06-20 12:36:00+00:00 strategy [INFO] SELL at 9785.00
2012-06-20 12:49:00+00:00 strategy [INFO] BUY at 9794.00
2012-06-20 12:58:00+00:

2012-07-06 13:52:00+00:00 strategy [INFO] SELL at 10642.00
2012-07-06 13:58:00+00:00 strategy [INFO] BUY at 10666.00
2012-07-06 14:15:00+00:00 strategy [INFO] SELL at 10635.00
2012-07-09 11:05:00+00:00 strategy [INFO] BUY at 10580.00
2012-07-09 11:10:00+00:00 strategy [INFO] SELL at 10574.00
2012-07-09 11:20:00+00:00 strategy [INFO] BUY at 10582.55
2012-07-09 11:23:00+00:00 strategy [INFO] SELL at 10574.00
2012-07-09 11:33:00+00:00 strategy [INFO] BUY at 10579.95
2012-07-09 11:36:00+00:00 strategy [INFO] SELL at 10556.00
2012-07-09 11:53:00+00:00 strategy [INFO] BUY at 10552.40
2012-07-09 12:06:00+00:00 strategy [INFO] SELL at 10542.00
2012-07-09 12:32:00+00:00 strategy [INFO] BUY at 10543.95
2012-07-09 12:54:00+00:00 strategy [INFO] SELL at 10540.00
2012-07-09 13:04:00+00:00 strategy [INFO] BUY at 10559.00
2012-07-09 13:16:00+00:00 strategy [INFO] SELL at 10548.00
2012-07-09 13:38:00+00:00 strategy [INFO] BUY at 10552.70
2012-07-09 13:50:00+00:00 strategy [INFO] SELL at 10555.05
2012-

2012-07-25 11:12:00+00:00 strategy [INFO] SELL at 10251.55
2012-07-25 11:28:00+00:00 strategy [INFO] BUY at 10259.90
2012-07-25 11:44:00+00:00 strategy [INFO] SELL at 10262.00
2012-07-25 11:49:00+00:00 strategy [INFO] BUY at 10262.50
2012-07-25 12:19:00+00:00 strategy [INFO] SELL at 10264.50
2012-07-25 12:33:00+00:00 strategy [INFO] BUY at 10281.00
2012-07-25 13:00:00+00:00 strategy [INFO] SELL at 10296.50
2012-07-25 13:10:00+00:00 strategy [INFO] BUY at 10309.60
2012-07-25 13:18:00+00:00 strategy [INFO] SELL at 10300.00
2012-07-25 13:42:00+00:00 strategy [INFO] BUY at 10282.40
2012-07-25 14:05:00+00:00 strategy [INFO] SELL at 10308.50
2012-07-26 11:13:00+00:00 strategy [INFO] BUY at 10258.00
2012-07-26 11:26:00+00:00 strategy [INFO] SELL at 10252.05
2012-07-26 11:36:00+00:00 strategy [INFO] BUY at 10256.95
2012-07-26 11:55:00+00:00 strategy [INFO] SELL at 10256.20
2012-07-26 12:27:00+00:00 strategy [INFO] BUY at 10250.10
2012-07-26 12:40:00+00:00 strategy [INFO] SELL at 10240.00
2012-

2012-08-14 11:20:00+00:00 strategy [INFO] BUY at 10471.90
2012-08-14 11:29:00+00:00 strategy [INFO] SELL at 10451.45
2012-08-14 11:40:00+00:00 strategy [INFO] BUY at 10471.95
2012-08-14 11:50:00+00:00 strategy [INFO] SELL at 10463.70
2012-08-14 11:57:00+00:00 strategy [INFO] BUY at 10469.50
2012-08-14 12:37:00+00:00 strategy [INFO] SELL at 10581.00
2012-08-14 12:57:00+00:00 strategy [INFO] BUY at 10581.00
2012-08-14 13:12:00+00:00 strategy [INFO] SELL at 10581.30
2012-08-14 13:20:00+00:00 strategy [INFO] BUY at 10587.00
2012-08-14 13:43:00+00:00 strategy [INFO] SELL at 10600.00
2012-08-16 11:23:00+00:00 strategy [INFO] BUY at 10548.00
2012-08-16 11:46:00+00:00 strategy [INFO] SELL at 10548.00
2012-08-16 11:53:00+00:00 strategy [INFO] BUY at 10560.10
2012-08-16 12:22:00+00:00 strategy [INFO] SELL at 10562.10
2012-08-16 12:55:00+00:00 strategy [INFO] BUY at 10541.50
2012-08-16 13:05:00+00:00 strategy [INFO] SELL at 10529.60
2012-08-16 13:21:00+00:00 strategy [INFO] BUY at 10523.45
2012-0

...

2016-09-19 12:35:00+00:00 strategy [INFO] BUY at 20010.00
2016-09-19 12:45:00+00:00 strategy [INFO] SELL at 19983.20
2016-09-19 12:56:00+00:00 strategy [INFO] BUY at 19992.90
2016-09-19 13:21:00+00:00 strategy [INFO] SELL at 19993.00
2016-09-19 13:50:00+00:00 strategy [INFO] BUY at 19980.00
2016-09-19 13:59:00+00:00 strategy [INFO] SELL at 19973.10
2016-09-21 11:36:00+00:00 strategy [INFO] BUY at 19990.05
2016-09-21 11:51:00+00:00 strategy [INFO] SELL at 19990.00
2016-09-21 12:34:00+00:00 strategy [INFO] BUY at 19966.20
2016-09-21 12:56:00+00:00 strategy [INFO] SELL at 19967.00
2016-09-21 13:23:00+00:00 strategy [INFO] BUY at 19957.10
2016-09-21 13:34:00+00:00 strategy [INFO] SELL at 19929.65
2016-09-21 13:52:00+00:00 strategy [INFO] BUY at 19938.95
2016-09-21 14:02:00+00:00 strategy [INFO] SELL at 19926.00
2016-09-22 12:02:00+00:00 strategy [INFO] BUY at 20081.95
2016-09-22 12:15:00+00:00 strategy [INFO] SELL at 20088.95
2016-09-22 12:17:00+00:00 strategy [INFO] BUY at 20094.60
2016-0

2016-10-10 11:16:00+00:00 strategy [INFO] SELL at 19525.95
2016-10-10 11:51:00+00:00 strategy [INFO] BUY at 19512.00
2016-10-10 12:13:00+00:00 strategy [INFO] SELL at 19515.00
2016-10-10 12:44:00+00:00 strategy [INFO] BUY at 19492.95
2016-10-10 12:48:00+00:00 strategy [INFO] SELL at 19480.00
2016-10-10 13:20:00+00:00 strategy [INFO] BUY at 19468.00
2016-10-10 13:44:00+00:00 strategy [INFO] SELL at 19453.35
2016-10-13 11:24:00+00:00 strategy [INFO] BUY at 19159.60
2016-10-13 11:32:00+00:00 strategy [INFO] SELL at 19130.05
2016-10-13 11:58:00+00:00 strategy [INFO] BUY at 19130.00
2016-10-13 12:05:00+00:00 strategy [INFO] SELL at 19122.80
2016-10-13 12:15:00+00:00 strategy [INFO] BUY at 19126.40
2016-10-13 12:32:00+00:00 strategy [INFO] SELL at 19129.95
2016-10-13 13:33:00+00:00 strategy [INFO] BUY at 19015.55
2016-10-13 13:39:00+00:00 strategy [INFO] SELL at 19006.35
2016-10-14 11:21:00+00:00 strategy [INFO] BUY at 19024.85
2016-10-14 11:54:00+00:00 strategy [INFO] SELL at 19040.80
2016-

2016-11-02 11:08:00+00:00 strategy [INFO] BUY at 19248.25
2016-11-02 11:34:00+00:00 strategy [INFO] SELL at 19257.10
2016-11-02 11:47:00+00:00 strategy [INFO] BUY at 19270.20
2016-11-02 11:58:00+00:00 strategy [INFO] SELL at 19242.05
2016-11-02 12:11:00+00:00 strategy [INFO] BUY at 19264.40
2016-11-02 12:53:00+00:00 strategy [INFO] SELL at 19320.00
2016-11-02 12:55:00+00:00 strategy [INFO] BUY at 19333.10
2016-11-02 13:05:00+00:00 strategy [INFO] SELL at 19305.05
2016-11-02 13:16:00+00:00 strategy [INFO] BUY at 19324.90
2016-11-02 13:32:00+00:00 strategy [INFO] SELL at 19312.00
2016-11-02 13:57:00+00:00 strategy [INFO] BUY at 19320.15
2016-11-02 14:10:00+00:00 strategy [INFO] SELL at 19295.10
2016-11-03 11:06:00+00:00 strategy [INFO] BUY at 19349.00
2016-11-03 11:17:00+00:00 strategy [INFO] SELL at 19333.50
2016-11-03 11:49:00+00:00 strategy [INFO] BUY at 19310.00
2016-11-03 12:09:00+00:00 strategy [INFO] SELL at 19307.80
2016-11-03 12:24:00+00:00 strategy [INFO] BUY at 19326.10
2016-1

2016-11-22 12:46:00+00:00 strategy [INFO] SELL at 18377.25
2016-11-22 12:56:00+00:00 strategy [INFO] BUY at 18429.70
2016-11-22 13:07:00+00:00 strategy [INFO] SELL at 18389.80
2016-11-22 13:30:00+00:00 strategy [INFO] BUY at 18415.00
2016-11-22 13:49:00+00:00 strategy [INFO] SELL at 18400.50
2016-11-23 11:31:00+00:00 strategy [INFO] BUY at 18570.00
2016-11-23 11:44:00+00:00 strategy [INFO] SELL at 18558.30
2016-11-23 12:06:00+00:00 strategy [INFO] BUY at 18548.00
2016-11-23 12:13:00+00:00 strategy [INFO] SELL at 18529.75
2016-11-23 12:36:00+00:00 strategy [INFO] BUY at 18530.15
2016-11-23 13:13:00+00:00 strategy [INFO] SELL at 18530.20
2016-11-23 13:29:00+00:00 strategy [INFO] BUY at 18552.30
2016-11-23 14:10:00+00:00 strategy [INFO] SELL at 18580.00
2016-11-24 11:07:00+00:00 strategy [INFO] BUY at 18390.55
2016-11-24 11:13:00+00:00 strategy [INFO] SELL at 18374.20
2016-11-24 11:25:00+00:00 strategy [INFO] BUY at 18389.40
2016-11-24 11:40:00+00:00 strategy [INFO] SELL at 18381.65
2016-

2016-12-09 13:59:00+00:00 strategy [INFO] BUY at 18704.00
2016-12-09 14:08:00+00:00 strategy [INFO] SELL at 18695.00


In [7]:
print("Final portfolio value: %.2f" % (crossover.getBroker().getEquity()))

Final portfolio value: 72870.55


In [8]:
from pyalgotrade.stratanalyzer import returns
from pyalgotrade.stratanalyzer import sharpe
from pyalgotrade.stratanalyzer import drawdown
from pyalgotrade.stratanalyzer import trades

In [11]:
retAnalyzer = returns.Returns()
SMACrossOver.attachAnalyzer(retAnalyzer)
sharpeRatioAnalyzer = sharpe.SharpeRatio()
SMACrossOver.attachAnalyzer(sharpeRatioAnalyzer)
drawDownAnalyzer = drawdown.DrawDown()
SMACrossOver.attachAnalyzer(drawDownAnalyzer)
tradesAnalyzer = trades.Trades()
SMACrossOver.attachAnalyzer(tradesAnalyzer)

NameError: name 'self' is not defined

In [13]:
print("Final portfolio value: %.2f" % SMACrossOver.getResult())

TypeError: getResult() missing 1 required positional argument: 'self'

In [14]:
print("Cumulative returns: %.2f %%" % (retAnalyzer.getCumulativeReturns()[-1] * 100))

IndexError: list index out of range

In [ ]:

from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade import plotter
import numpy as np
from pyalgotrade.barfeed import quandlfeed
import datetime
from pyalgotrade.technical import ma
filename = "bank_nifty1.csv"
import pytz
firstDate = datetime.datetime(2012,5,30,9,15,0,0,pytz.utc)
endDate = datetime.datetime(2016,12,9,15,29,0,0,pytz.utc)

from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade.technical import cross


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        strategy.BacktestingStrategy.__init__(self, feed, 100000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        #self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at %.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at %.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 2, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1]:
            self.__position.exitMarket()


def run_strategy(smaPeriod):
    
    feed = csvfeed.GenericBarFeed(1*Frequency.MINUTE,pytz.utc)
#feed = quandlfeed.Feed()
    feed.setBarFilter(csvfeed.DateRangeFilter(firstDate,endDate))
    feed.addBarsFromCSV('bank_nifty',filename)

    #myStrategy = MyStrategy(feed, "bank_nifty")
    #myStrategy.run()
    crossover = MyStrategy(feed,"bank_nifty",smaPeriod)
    #plt = plotter.StrategyPlotter(MyStrategy)
    crossover.run()
    #plt.plot()
    print("Final portfolio value: %.2f" % (MyStrategy.getBroker().getEquity()))

run_strategy(15)